In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.cluster import KMeans



In [2]:

# オートエンコーダの定義
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [5]:

# カスタムデータセットの定義
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

path_indir = '/mnt/g/20231222_thunder_spoeffi_YSIY1475_YSIY944_TNY168/Project002_YSIY1475_with-IAA-at6hr/crop_Image001_ch01'
dataset = ImageFolder(root=path_indir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


FileNotFoundError: Couldn't find any class folder in /mnt/g/20231222_thunder_spoeffi_YSIY1475_YSIY944_TNY168/Project002_YSIY1475_with-IAA-at6hr/crop_Image001_ch01.

In [5]:

# オートエンコーダのトレーニング
autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for data in dataloader:
        images, _ = data
        optimizer.zero_grad()
        outputs = autoencoder(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()


2024-01-05 17:43:31.697657: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-01-05 17:43:32.713764: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-01-05 17:43:33.559598: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-01-05 17:43:33.672341: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/10


ValueError: in user code:

    File "/home/yeakiest/anaconda3/envs/cellpose/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/yeakiest/anaconda3/envs/cellpose/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/yeakiest/anaconda3/envs/cellpose/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/yeakiest/anaconda3/envs/cellpose/lib/python3.8/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/yeakiest/anaconda3/envs/cellpose/lib/python3.8/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/home/yeakiest/anaconda3/

In [ ]:

# エンコーダを使用して中間表現を取得
encoded_features = []
with torch.no_grad():
    for data in dataloader:
        images, _ = data
        encoded = autoencoder.encoder(images)
        encoded_features.append(encoded.view(encoded.size(0), -1).numpy())

encoded_features = torch.cat(encoded_features)


In [ ]:

# k-meansクラスタリング
kmeans = KMeans(n_clusters=2)  # クラスタ数は適切に設定してください
kmeans.fit(encoded_features.numpy())

# クラスタリング結果
labels = kmeans.labels_
print(labels)
